"Extractive summarization" oзначава да намерим важни части в текста, и да ги съберем като резюме на текста дословно. "abstractive summarization" пресъздава важната част на текста по нов начин след интерпретация и изследването му чрез NLP техники. Документа е насочен към "Extractive summarization" и ще приложим две техники за резюмиране Latent Semantic Analysis и Text Rank. Първо ще разгледаме стандарните стъпки при "Extractive summarization"

# Dataset 

Използваме newsroom dataset, при който са събирани статии години наред заедно като имаме налични и резюметата на статиите. Използваме първите 1000 елемента oт тестовия dataset.

[Newsroom](https://github.com/lil-lab/newsroom)

# Стъпка 1

Създаване на междинна репрезентация на входния текст. 
Има два подхода: topic representation и indicator representation.

При topic representation намираме важните теми чрез SVD, докато при indicator representation oписваме изречения спрямо различни фактори: при TextRank е cos оценката между две изречения.

# Стъпка 2
Оценяме изречения спрямо избрания подход. Всяко изречение получава оценка. 
При topic representation оценката ни дава колко изречението обяснява темите в текста. При indicator representation оценката е просто агрегацията на избраните от нас фактори (ние ще имаме един)


# Стъпка 3
Избираме най-важните k изречения спрямо оценките, които формират нашето резюме

# Подходи

# Latent Semantic Analysis (LSA)

LSA e unsupervised метод. Първата стъпка е да създадем term-sentence matrix , където имаме всеки term от документа като ред и всяко изречение като стълб. Всяка клетка се попълва с теглото да думата в изречение с TF-IDF оцента После използваме - singular value decomposition (SVD), която трансформира матрица в три матрици term-topic matrix, diagonal matrix, topic-sentence matrix. След това се използват различни техники за избиране на изречение. Тук е използвана техниката на "Gong and Liu".


# Теxt Rank

Вдъхновен от PageRank този метод представя изреченията като свързан граф, където, на база оценка колко близки са две изречение, създаваме връзки между два върха на графа. Близостта между две изречения се измерва с cos оценка. Изречения, към които сочат повече други изречения най-вероятно са по-важни и трябва да участват в резюмето. Този метод не е специфичен за език и може да се и използва в различни езици.

# Eвристичен подход 

Вземаме първите k изречения на текста при максимална стойност 3. Този метод е само за ориентиране накъде се движим с основните методи.

# Oпределяне на k 

Спрямо наличния dataset забелязваме че преобладават по кратки резюметата (1 изречение) дори и за по големите текстове. Спираме се на по консервативен подход за определяне на k - log10(Брой изречения) при стойност на минимална стойност на k >= 1.

# Оценки
Между предоставеното резюме и кандидат резюмето
- cos oценка
- Rouge L оценка намираща най-дългата подредица

# Eксперименти

Използваме първите 1000 елемента от dataset newsroom. 
За всяко получено резюме, сравняваме с cos и rougeL оценките 
спрямо наличното ни. Изчисляваме средната оценка и медианата на получени резултати.  Използваме трите описани подхода.

## text-rank

### cos:
- avg: 0.25
- median: 0.2

### rouge
- avg: 0.16
- median: 0.08


## lsa

### cos
- avg: 0.26
- median: 0.15

### rouge
- avg: 0.15
- median 0.05

## heauristics:

### cos:
- avg: 0.33
- median: 0.22

### rouge:
- avg: 0.22
- median: 0.09


# Бъдеща разработка

- Фокусиране върху специфична област (пример медицински новини)
- Избиране на различни оценъчни техники за изречение и теми за LSA
- Използване на Rouge-1, Rouge-2 и Rouge-N като допълнителни оценки
- Разшираване на списъка с абревиатури

Връзки:

https://www.researchgate.net/publication/220195824_Text_summarization_using_Latent_Semantic_Analysis

https://medium.com/sciforce/towards-automatic-text-summarization-extractive-methods-e8439cd54715

In [8]:
# Utils functions

import numpy as np
from decimal import *
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import os.path
from gensim import corpora
import nltk
import json

# nltk.download('stopwords')

tokenizer = RegexpTokenizer(r'\w+')
en_stop = set(stopwords.words('english'))
p_stemmer = PorterStemmer()


def load_data(count):
    documents_list = []
    summaries = []
    with open('1000.json') as file:
        string = file.read()
        entries = json.loads(string)
        i = 0
        for entry in entries:
            i = i + 1
            documents_list.append(entry["text"])
            summaries.append(entry["summary"])
            if i == count:
                break
    return documents_list, summaries


def get_stemmed(doc_to_stem):
    if doc_to_stem is not None:
        raw = doc_to_stem.lower()
        tokens = tokenizer.tokenize(raw)
        stopped_tokens = [a for a in tokens if not a in en_stop]
        return [p_stemmer.stem(j) for j in stopped_tokens]
    else:
        return []


def count_tokens(sent, topic):
    sum = 0
    stemmed_tokens = get_stemmed(sent)
    for token in stemmed_tokens:
        if token == topic:
            sum = sum + 1
    return sum


def calculate_vec(first_doc, second_doc):
    first_stem = get_stemmed(first_doc)
    second_stem = get_stemmed(second_doc)
    if len(first_stem) == 0 or len(second_stem) == 0:
        return 0.0
    total = set(first_stem).union(set(second_stem))
    word_dict_first = dict.fromkeys(total, 0)
    word_dict_second = dict.fromkeys(total, 0)
    for word in first_stem:
        if word in word_dict_first:
            word_dict_first[word] += 1

    for word in second_stem:
        if word in word_dict_second:
            word_dict_second[word] += 1
    a = list(word_dict_first.values())
    b = list(word_dict_second.values())
    cos_sim = dot(a, b) / (norm(a) * norm(b))
    return  cos_sim

def rougeL(candidate, reference):
    candidate_stem = get_stemmed(candidate)
    reference_stem = get_stemmed(reference)
    if len(candidate_stem) == 0 or len(reference_stem) == 0:
        return 0.0
    current_sequence = 0
    max_subsequence = 0
    for i in range(len(candidate_stem)):
        for j in range(len(reference_stem)):
            if j + i < len(candidate_stem) and (candidate_stem[j + i] == reference_stem[j]):
                current_sequence += 1
            else:
                if current_sequence > max_subsequence:
                    max_subsequence = current_sequence
                current_sequence = 0
    return max_subsequence/len(reference_stem)

In [12]:
# Latent Semantic Analysis(LSA)

# Latent Semantic Analysis is an algebraic-statistical method that extracts hidden semantic structures of words and sentences. It is an unsupervised approach that does not need any training or external knowledge. LSA uses the context of
# the input document and extracts information such as which words are used together and which common words are seen
# in different sentences. A high number of common words among sentences indicates that the sentences are semantically
# related. The meaning of a sentence is decided using the words it contains, and meanings of words are decided using the
# sentences that contains the words. Singular Value Decomposition, an algebraic method, is used to find out the interrelations between sentences and words. Besides having the capability of modelling relationships among words and sentences, SVD has the capability of noise reduction, which helps to improve accuracy. In order to see how LSA can
# represent the meanings of words and sentences.


# 3.1.1. Step 1
# Input matrix creation: an input document needs to be represented in a way that enables a computer to understand and
# perform calculations on it. This representation is usually a matrix representation where columns are sentences and rows
# are words/phrases. The cells are used to represent the importance of words in sentences. Different approaches can be
# used for filling out the cell values. Since all words are not seen in all sentences, most of the time the created matrix is
# sparse.
# The way in which an input matrix is created is very important for summarization, since it affects the resulting
# matrices calculated with SVD. As already mentioned, SVD is a complex algorithm and its complexity increases with
# the size of input matrix, which degrades the performance. In order to reduce the matrix size, rows of the matrix, i.e. the
# words, can be reduced by approaches like removing stop words, using the roots of words only, using phrases instead of
# words and so on. Also, cell values of matrix can change the results of SVD. There are different approaches to filling out
# the cell values. These approaches are as follows.
# • Frequency of word: the cell is filled in with the frequency of the word in the sentence.
# • Binary representation: the cell is filled in with 0/1 depending on the existence of a word in the sentence.
# • Tf-idf (Term Frequency-Inverse Document Frequency): the cell is filled in with the tf-idf value of the word.
# A higher tf-idf value means that the word is more frequent in the sentence but less frequent in the whole
# document. A higher value also indicates that the word is much more representative for that sentence than
# others.
# • Log entropy: the cell is filled in with the log-entropy value of the word, which gives information on how informative the word is in the sentence.
# • Root type: the cell is filled in with the frequency of the word if its root type is a noun, otherwise the cell value
# is set to 0.
# • Modified Tf-idf: this approach is proposed in Ozsoy et al. [3], in order to eliminate noise from the input matrix.
# The cell values are set to tf-idf scores first, and then the words that have scores less than or equal to the average
# of the row are set to 0.

import pandas as pd
from collections import Counter
import numpy as np
from collections import defaultdict
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import decimal
import math

document_list, summaries = load_data(2)


def get_idf_dataframe(sentenses):
    document_frequency = defaultdict(lambda: 0)
    sentenses_stemmed = defaultdict()
    for sent in sentenses:
        stemmed_tokens = get_stemmed(sent)
        sentenses_stemmed[sent] = stemmed_tokens
        for _word in set(stemmed_tokens):
            document_frequency[_word] += 1

    idf_df = pd.DataFrame(list(document_frequency.items()), columns=['word', 'doc_freq'])
    idf_df['idf'] = np.log10(len(sentenses)/idf_df['doc_freq'])
    idf_df.sort_values(by=['idf'], inplace=True)
    return idf_df, sentenses_stemmed


def tfidf(word, sent, idf_df, sentenses_stemmed):
    sent_tokens = sentenses_stemmed[sent]
    count = sent_tokens.count(word)
    tf = (1 + np.log10(count)) if count != 0 else 1
    inner_idf = idf_df[idf_df['word'] == word].iloc[0]['idf']
    return  tf * inner_idf


# 3.2.1. Gong and Liu (2001). The algorithm of Gong and Liu [4] is one of the main studies conducted in LSA-based text
# summarization. After representing the input document in the matrix and calculating SVD values, VT matrix, the matrix
# Figure 1. LSA can represent the meaning of words and sentences.
# of extracted concepts × sentences is used for selecting the important sentences. In VT matrix, row order indicates the
# importance of the concepts, such that the first row represents the most important concept extracted. The cell values of
# this matrix show the relation between the sentence and the concept. A higher cell value indicates that the sentence is
# more related to the concept.
# In the approach of Gong and Liu, one sentence is chosen from the most important concept, and then a second sentence is chosen from the second most important concept until a predefined number of sentences are collected. The number of sentences to be collected is given as a parameter.
# In Example 1, three sentences were given, and the SVD calculations were performed accordingly. The resulting VT
# matrix having rank set to two is given in Figure 2. In this figure, first, the concept con0 is chosen, and then the sentence
# sent1 is chosen, since it has the highest cell value in that row.
# The approach of Gong and Liu has some disadvantages that are defined by Steinberger and Jezek [5]. The first disadvantage is that the number of sentences to be collected is the same with the reduced dimension. If the given predefined
# number is large, sentences from less significant concepts are chosen. The second disadvantage is related to choosing
# only one sentence from each concept. Some concepts, especially important ones, can contain sentences that are highly
# related to the concept, but do not have the highest cell value. The last disadvantage is that all chosen concepts are
# assumed to be in the same importance level, which may not be true.
def get_summary(sentenses, VT):
    log_size = np.log10(len(sentenses))
    max_items = int(log_size) if log_size >= 1.0 else 1
    summary_elements = []
    for k in range(max_items):
        values = VT[k].round(15)
        max_index = 0
        max_value = values[0]
        for i in range(1, len(values)):
            value = values[i]
            if value > max_value:
                max_value = value
                max_index = i
        summary_elements.append(max_index)

    summary_elements.sort()

    return "".join([sentenses[el] for el in summary_elements])


scores = np.array([])
rouges = np.array([])

sent_tokenizer = PunktSentenceTokenizer()
punkt_param = PunktParameters()
punkt_param.abbrev_types = set(['dr', 'vs', 'mr', 'mrs', 'prof', 'inc', 'ms', 'u.s', 'rep'])
sentence_splitter = PunktSentenceTokenizer(punkt_param)

for i in range(len(document_list)):
    doc = document_list[i]
    terms = get_stemmed(doc)
    sentenses = sentence_splitter.tokenize(doc)
    if len(sentenses) == 0:
        continue
    idf_df, sentenses_stemmed = get_idf_dataframe(sentenses)
    term_doc_matrix = [[tfidf(term, sent, idf_df, sentenses_stemmed) for sent in sentenses] for term in terms]
    # 3.1.2. Step 2
    # Singular Value Decomposition: SVD is an algebraic method that can model relationships
    # among words/phrases and sentences. In this method, the given input matrix
    # A is decomposed into three new matrices as follows:
    U, S, VT = np.linalg.svd(term_doc_matrix)
    summary = get_summary(sentenses, VT)
    score = calculate_vec(summary, summaries[i])
    rouge1_score = rougeL(summary, summaries[i])
    print('=====doc======')
    # print(doc)
    print(i)
    print('=====real=======')
    print(summaries[i])
    print('======mine======')
    print(summary)
    print('======vec=======')
    print(score)
    if math.isnan(score) is False:
        scores = np.append(scores, score)
    print('====rouge1==========')
    print(rouge1_score)
    if math.isnan(rouge1_score) is False:
        rouges = np.append(rouges, rouge1_score)
    print('====================')


print('=====cos avg and median=======')
scores = scores[np.logical_not(np.isnan(scores))]
print(scores)
print(np.average(scores))
print(np.median(scores))
print('=====rouge avg and median=======')
rouges = rouges[np.logical_not(np.isnan(rouges))]
print(rouges)
print(np.average(rouges))
print(np.median(rouges))
print('====================')

=====doc======
0
=====real=======
Think the economy is bouncing back quickly?
======mine======
Think the economy is bouncing back quickly?
======vec=======
0.9999999999999998
====rouge1==========
1.0
=====doc======
1
=====real=======
Court documents unsealed Wednesday revealed much about Bruce Ivins' activities but still left questions unanswered.
======mine======
By Donna Leinwand, Ken Dilanian, Steve Sternberg and Dan Vergano, USA TODAY

WASHINGTON  On several nights before the anthrax attacks in September and October 2001, bioweapons scientist Bruce Ivins repeatedly spent long periods alone in a secure laboratory that housed a strain of the lethal bacteria.
======vec=======
0.09656090991705353
====rouge1==========
0.15384615384615385
=====cos avg and median=======
[1.         0.09656091]
0.5482804549585266
0.5482804549585266
=====rouge avg and median=======
[1.         0.15384615]
0.5769230769230769
0.5769230769230769


In [13]:
# Text Rank
import os.path
from gensim import corpora
from gensim.models import LsiModel
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import matplotlib.pyplot as plt
import numpy as np
from decimal import *
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import math

# Influenced by PageRank algorithm, these methods represent documents as a connected graph,
# where sentences form the vertices and edges between the sentences indicate how similar
# the two sentences are. The similarity of two sentences is measured with the help of
# cosine similarity with TFIDF weights for words and if it is greater than a certain threshold,
# these sentences are connected. This graph representation results in two outcomes:
# the sub-graphs included in the graph create topics covered in the documents,
# and the important sentences are identified. Sentences that are connected to
# many other sentences in a sub-graph are likely to be the center of the graph and
# will be included in the summary Since this method do not need language-specific
# linguistic processing, it can be applied to various languages [43].
# At the same time, such measuring only of the formal side of the sentence
# structure without the syntactic and semantic information limits the application of the method.

documents_list, summaries = load_data(2)

# Add connection counts to the
def get_connections_dictionary(sentenses):
    size = len(sentenses)
    d = dict.fromkeys(sentenses, 0)
    for i in range(size):
        current_sentence = sentenses[i]
        # We are searching only for one match
        # We are not supporting multiple max measures
        # or n-references to
        max_measured = 0
        max_index = 0
        for j in range(size):
            if i != j:
                measured_against = sentenses[j]
                # TODO: Use tf-idf
                cos = calculate_vec(current_sentence, measured_against)
                if cos > max_measured:
                    max_measured = cos
                    max_index = j

        # We are adding count to the max score for measured sentence
        # because the current is pointing to the measured
        d[sentenses[max_index]] += 1
    return d


def get_mine_summary(d, sentences):
    size = len(sentences)
    # Custom measure max sentences to get for summary
    max_size_float = np.log10(size)
    max_items = int(max_size_float) if max_size_float >= 1.0 else 1
    values = list(d.values())
    enumerated = [(v, i) for i, v in enumerate(values)]
    enumerated.sort(key=lambda tup: tup[0])
    enumerated.reverse()
    summary_elements = enumerated[0:max_items]
    return "".join([ sentenses[el[1]] for el in summary_elements])


sent_tokenizer = PunktSentenceTokenizer()
punkt_param = PunktParameters()
punkt_param.abbrev_types = set(['dr', 'vs', 'mr', 'mrs', 'prof', 'inc', 'ms', 'u.s', 'rep'])
sentence_splitter = PunktSentenceTokenizer(punkt_param)

scores = np.array([])
rouges = np.array([])

for i in range(len(documents_list)):
    doc = documents_list[i]
    sentenses = sentence_splitter.tokenize(doc)
    d = get_connections_dictionary(sentenses)
    summary = get_mine_summary(d, sentenses)
    score = calculate_vec(summary, summaries[i])
    rouge1_score = rougeL(summary, summaries[i])
    print('=====doc======')
    print(doc)
    print(i)
    print('=====real=======')
    print(summaries[i])
    print('======mine======')
    print(summary)
    print('======vec=======')
    print(score)
    if math.isnan(score) is False:
        scores = np.append(scores, score)
    print('====rouge1==========')
    print(rouge1_score)
    if math.isnan(rouge1_score) is False:
        rouges = np.append(rouges, rouge1_score)
    print('====================')

print('=====cos avg and median=======')
scores = scores[np.logical_not(np.isnan(scores))]
print(scores)
print(np.average(scores))
print(np.median(scores))
print('=====rouge avg and median=======')
rouges = rouges[np.logical_not(np.isnan(rouges))]
print(rouges)
print(np.average(rouges))
print(np.median(rouges))
print('====================')

=====doc======
Think the economy is bouncing back quickly? Think again, says the former top number-cruncher in charge of the Washington Bureau of Labor Statistics.

Keith Hall tells the New York Post Thursday that the BLS, White House and media are wasting time focusing on an edited set of data and using it to paint an incorrect picture of the American jobs market on the mend. The current U.S. unemployment level is reported to be around 7.6 percent.

“Right now [it’s] misleadingly low,” Hall told the New York paper.

Hall, like many other economists, believes that the more accurate reading of Americans who want a job but can’t find one is north of 10 percent.

Hall is now a senior research fellow at the Mercatus Center at George Mason University.

He says the jobless rate that grabs the most headlines -- called the U-3 -- doesn’t factor in people who have stopped looking for work, but does count employed people who have clocked in as little as an hour of work during the prior month.

H

In [14]:
# Heuristics approach
import pandas as pd
from collections import Counter
import numpy as np
from collections import defaultdict
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import decimal
import math

document_list, summaries = load_data(5)

sent_tokenizer = PunktSentenceTokenizer()
punkt_param = PunktParameters()
punkt_param.abbrev_types = set(['dr', 'vs', 'mr', 'mrs', 'prof', 'inc', 'ms', 'u.s', 'rep'])
sentence_splitter = PunktSentenceTokenizer(punkt_param)

size = len(document_list)

scores = np.array([])
rouges = np.array([])

for i in range(size):
    print('=====doc======')
    print(i)
    doc = document_list[i]
    sentenses = sentence_splitter.tokenize(doc)
    print(sentenses)
    log_size = np.log10(len(sentenses))
    max_items = int(log_size) if log_size >= 1.0 else 1
    summary = ''
    if len(sentenses) == 0:
        summary = ''
    elif max_items == 1:
        summary = sentenses[0]
    elif max_items == 2:
        summary = "".join(sentenses[:2])
    else:
        summary = "".join(sentenses[:3])
    score = calculate_vec(summary, summaries[i])
    rougeL_score = rougeL(summary, summaries[i])

    print('=====real=======')
    print(summaries[i])
    print('======mine======')
    print(summary)
    print('======vec=======')
    print(score)
    if math.isnan(score) is False:
        scores = np.append(scores, score)
    print('====rouge1==========')
    print(rougeL_score)
    if math.isnan(rougeL_score) is False:
        rouges = np.append(rouges, rougeL_score)
    print('====================')


print('=====cos avg and median=======')
scores = scores[np.logical_not(np.isnan(scores))]
print(scores)
print(np.average(scores))
print(np.median(scores))
print('=====rouge avg and median=======')
rouges = rouges[np.logical_not(np.isnan(rouges))]
print(rouges)
print(np.average(rouges))
print(np.median(rouges))
print('====================')


=====doc======
0
['Think the economy is bouncing back quickly?', 'Think again, says the former top number-cruncher in charge of the Washington Bureau of Labor Statistics.', 'Keith Hall tells the New York Post Thursday that the BLS, White House and media are wasting time focusing on an edited set of data and using it to paint an incorrect picture of the American jobs market on the mend.', 'The current U.S. unemployment level is reported to be around 7.6 percent.', '“Right now [it’s] misleadingly low,” Hall told the New York paper.', 'Hall, like many other economists, believes that the more accurate reading of Americans who want a job but can’t find one is north of 10 percent.', 'Hall is now a senior research fellow at the Mercatus Center at George Mason University.', 'He says the jobless rate that grabs the most headlines -- called the U-3 -- doesn’t factor in people who have stopped looking for work, but does count employed people who have clocked in as little as an hour of work during